In [3]:
 # %pip install robin_stocks openai==1.2.2 yfinance requests_oauthlib twilio boto3 gnews pymongo --upgrade 

In [4]:
import robin_stocks.robinhood as r
import requests
import json
import pandas as pd
import datetime
from dateutil import parser
from dateutil.relativedelta import relativedelta
from dateutil.tz import tzlocal
from datetime import timedelta
import pytz
import boto3
from tqdm.auto import tqdm
from io import StringIO, BytesIO

In [5]:
from news2roi import News2ROI

In [6]:
s3 = boto3.client('s3')
bucket_name = 'jvalansi-notebooks-data'

# s3.upload_file("/root/.aws/secrets.json", bucket_name, ".aws.json")
# s3.upload_file("/root/.rapidapi.json", bucket_name, ".rapidapi.json")
# s3.upload_file("/root/.newsapi.json", bucket_name, ".newsapi.json")
# s3.upload_file("/root/.robinhood.json", bucket_name, ".robinhood.json")

data = s3.get_object(Bucket=bucket_name, Key=".aws.json")
j = json.loads(data['Body'].read().decode('utf8'))
openai_key = j['OPENAI_KEY']
data = s3.get_object(Bucket=bucket_name, Key=".rapidapi.json")
# data = s3.get_object(Bucket=bucket_name, Key=".newsapi.json")
j = json.loads(data['Body'].read().decode('utf8'))
news_key = j["key"]
data = s3.get_object(Bucket=bucket_name, Key=".robinhood.json")
trade_cred = json.loads(data['Body'].read().decode('utf8'))
data = s3.get_object(Bucket=bucket_name, Key=".twilio.json")
notify_cred = json.loads(data['Body'].read().decode('utf8'))

In [7]:
n2r = News2ROI(openai_key, news_key, trade_cred, notify_cred)

In [8]:
date = datetime.datetime.today().strftime('%Y-%m-%d')

In [7]:
# date = "2024-01-26"

In [9]:
# get new news articles
articles = n2r.get_news(date=date, source='google')

In [10]:
len(articles)

38

In [10]:
articles = [article for article in articles if News2ROI.contains_words(article, words={'United States',"US"})]

In [11]:
len(articles)

38

In [12]:
articles[0]

{'title': 'New York Special Election: Tom Suozzi Wins House Seat Vacated by George Santos - The New York Times',
 'description': "New York Special Election: Tom Suozzi Wins House Seat Vacated by George Santos  The New York TimesThe 'scandal penalty' that George Santos could impose on NY-03  POLITICOTrump rages at ‘foolish’ GOP candidate who lost race for Santos’s New York House seat  The HillDems flip seat as Suozzi wins crucial special congressional election in New York  Fox NewsTakeaways from New York’s high-stakes special election  CNN",
 'publishedAt': 'Wed, 14 Feb 2024 07:08:00 GMT',
 'url': 'https://news.google.com/rss/articles/CBMiTWh0dHBzOi8vd3d3Lm55dGltZXMuY29tLzIwMjQvMDIvMTMvbnlyZWdpb24vcGlsaXAtc3VvenppLXNhbnRvcy1lbGVjdGlvbi5odG1s0gEA?oc=5&hl=en-US&gl=US&ceid=US:en',
 'publisher': {'href': 'https://www.nytimes.com',
  'title': 'The New York Times'}}

In [13]:
sorted([article['publishedAt'] for article in articles])

['Tue, 13 Feb 2024 08:29:00 GMT',
 'Tue, 13 Feb 2024 10:18:45 GMT',
 'Tue, 13 Feb 2024 17:32:15 GMT',
 'Tue, 13 Feb 2024 17:42:21 GMT',
 'Tue, 13 Feb 2024 18:33:01 GMT',
 'Tue, 13 Feb 2024 19:37:58 GMT',
 'Tue, 13 Feb 2024 20:26:00 GMT',
 'Tue, 13 Feb 2024 20:59:00 GMT',
 'Tue, 13 Feb 2024 21:03:00 GMT',
 'Tue, 13 Feb 2024 21:25:00 GMT',
 'Tue, 13 Feb 2024 22:01:23 GMT',
 'Tue, 13 Feb 2024 22:03:15 GMT',
 'Tue, 13 Feb 2024 22:48:00 GMT',
 'Tue, 13 Feb 2024 22:50:33 GMT',
 'Tue, 13 Feb 2024 22:54:12 GMT',
 'Tue, 13 Feb 2024 23:36:00 GMT',
 'Tue, 13 Feb 2024 23:46:00 GMT',
 'Wed, 14 Feb 2024 00:10:00 GMT',
 'Wed, 14 Feb 2024 00:48:06 GMT',
 'Wed, 14 Feb 2024 01:13:11 GMT',
 'Wed, 14 Feb 2024 01:17:28 GMT',
 'Wed, 14 Feb 2024 01:37:00 GMT',
 'Wed, 14 Feb 2024 01:51:05 GMT',
 'Wed, 14 Feb 2024 01:54:00 GMT',
 'Wed, 14 Feb 2024 02:24:25 GMT',
 'Wed, 14 Feb 2024 02:42:32 GMT',
 'Wed, 14 Feb 2024 03:13:00 GMT',
 'Wed, 14 Feb 2024 03:26:38 GMT',
 'Wed, 14 Feb 2024 04:00:14 GMT',
 'Wed, 14 Feb 

In [19]:
articles_ = [article for article in articles if n2r.is_recent(article['publishedAt'], delta=datetime.timedelta(hours=2))]

In [20]:
len(articles_)

5

In [21]:
articles_[0]

{'title': 'New York Special Election: Tom Suozzi Wins House Seat Vacated by George Santos - The New York Times',
 'description': "New York Special Election: Tom Suozzi Wins House Seat Vacated by George Santos  The New York TimesThe 'scandal penalty' that George Santos could impose on NY-03  POLITICOTrump rages at ‘foolish’ GOP candidate who lost race for Santos’s New York House seat  The HillDems flip seat as Suozzi wins crucial special congressional election in New York  Fox NewsTakeaways from New York’s high-stakes special election  CNN",
 'publishedAt': 'Wed, 14 Feb 2024 07:08:00 GMT',
 'url': 'https://news.google.com/rss/articles/CBMiTWh0dHBzOi8vd3d3Lm55dGltZXMuY29tLzIwMjQvMDIvMTMvbnlyZWdpb24vcGlsaXAtc3VvenppLXNhbnRvcy1lbGVjdGlvbi5odG1s0gEA?oc=5&hl=en-US&gl=US&ceid=US:en',
 'publisher': {'href': 'https://www.nytimes.com',
  'title': 'The New York Times'}}

In [22]:
# get sentiment
res = []


In [24]:
for article in tqdm(articles_):
    res += [n2r.analyse_article(article)]
    

  0%|          | 0/5 [00:00<?, ?it/s]

02/14/2024 07:31:53 AM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
02/14/2024 07:32:02 AM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
02/14/2024 07:32:15 AM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
02/14/2024 07:32:28 AM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
02/14/2024 07:32:36 AM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [11]:
candidates = News2ROI.get_candidates(res, threshold=3)

NameError: name 'res' is not defined

In [26]:
candidates

,stock,ticker,exchange,reasoning,action,date,data
8,"Alaska Air Group, Inc.",ALK,NYSE,The authorization of a strike by Alaska Airlin...,3.0,"Wed, 14 Feb 2024 05:59:14 GMT",Alaska Airlines flight attendants authorize st...


In [36]:
# ticker = low_sentiment.iloc[0]['ticker']

In [27]:
candidate = candidates.iloc[0].copy(deep=True)

In [30]:
ticker = candidate['ticker']

In [29]:
ticker = 'aapl'

In [15]:
# get options


In [16]:
# expirationDate = the next friday

In [31]:
option_data = n2r.get_option_data(ticker)

Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data 

In [32]:
len(option_data)

15

In [33]:
option_df = News2ROI.parse_option_data(option_data)

In [ ]:
option_df

In [125]:
# option_df.plot(x='strike_price', y='adjusted_mark_price')

In [124]:
# option_df.plot(x='strike_price', y='break_even_price')

In [122]:
# find lowest option price with highest break even price
# usually the current stock price
# example stock costs 100
# option to put (sell) in a week for 100 costs 1$
# if the stock costs 99 in a week, I break even, anything below I earn
# e.g if the stock costs 98 in a week, I sell at 100, I get 2$, but it cost 1$, I made 1$

# if I buy the option to sell in 150, it costs 49.85
# if the stock costs 100 in a week, I sell at 150, I made 50, but it cost me 49.85, I made 0.15$

# if I buy the option to sell in 90, it costs 0.02
# if the stock costs 100 in a week, I sell in 90, I lost 10$

# options are at a multiples of 100s
# so if the stock costs 100
# and costs 4 to sell at 104
# if the stock goes to 101, it'll cost 3, so I could sell at 3$ and lose 100
# I could lose everythin
# If it goes to 99, it'll cost 5, and I could sell at 5, and gain 100
# I could 

In [74]:
# I want to maximize ROI
# I need to check which one maximizes for the expected value of 0.2% drop

In [34]:
row = option_df[option_df['normalized_gain']==option_df['normalized_gain'].max()].iloc[0].to_dict()

In [42]:
# candidate = pd.Series()

In [35]:
candidate['strike_price'] = row['strike_price']

candidate['buy_price'] = row['bid_price']

candidate['stock_buy_price'] = row['current_price']

In [36]:
row

{'type': 'put',
 'strike_price': 35.0,
 'adjusted_mark_price': 0.13,
 'break_even_price': 34.87,
 'volume': 140,
 'chance_of_profit_long': 0.142511,
 'ask_price': 0.15,
 'ask_size': 161,
 'bid_price': 0.1,
 'bid_size': 9,
 'pred_bid_price': 1.0,
 'normalized_gain': 2.2666666666666666,
 'current_price': 36.6}

In [28]:
# when the target price hits, the option prices should shift with the target price
# say current price is 100

In [ ]:
# maybe easier to calculate - each price shifts one up

In [ ]:
# place option for the day
# r.order_buy_option_limit

In [ ]:
# place reminder to sell tomorrow


In [ ]:
# sell previous reminders


In [139]:
# df = pd.DataFrame(candidate).T

In [39]:
def load_candidates(bucket_name):
    s3 = boto3.client('s3')
    data = s3.get_object(Bucket=bucket_name, Key="candidates.tsv")
    data = data['Body'].read().decode('utf8')
    df = pd.read_csv(StringIO(data), sep='\t', index_col=0)
    return df

In [13]:
df = News2ROI.load_candidates(bucket_name)

In [14]:
df

,stock,ticker,exchange,reasoning,action,date,data,strike_price,bid_price,current_price
0,Vista Outdoor Inc.,VSTO,NYSE,The news about California's ammunition backgro...,5.0,2024-02-06 03:08:50,California ammunition background check law can...,27.5,0.05,29.95
1,Vista Outdoor Inc.,VSTO,NYSE,The news about California's ammunition backgro...,5.0,2024-02-06 03:08:50,California ammunition background check law can...,27.5,0.05,29.95
2,"Smith & Wesson Brands, Inc.",SWBI,NASDAQ,News coverage of a shooting incident at the Ch...,3.0,2024-02-16 01:52:55,What to know about Kansas City's homicide rate...,12.5,0.40,13.95
3,"Smith & Wesson Brands, Inc.",SWBI,NASDAQ,The news article details a mass shooting event...,3.0,2024-02-16 04:14:00,Dispute may have led to the mass shooting afte...,12.5,0.40,13.95
4,Toyota Motor Corporation,TM,NYSE,"As a major Japanese multinational corporation,...",4.0,2024-02-16 00:36:00,Japan slipped into a technical recession. The ...,210.0,0.60,228.55
5,Dow Jones Industrial Average ETF,DIA,NYSE Arca,The Dow Jones Industrial Average (DJIA) often ...,3.0,2024-02-16 14:39:00,Dow falls as yields rise after another hot inf...,385.0,1.48,386.65
6,"Smith & Wesson Brands, Inc.",SWBI,NASDAQ,The news of a shooting incident during the Kan...,3.0,2024-02-16 18:19:44,2 juveniles charged in Kansas City Chiefs Supe...,10.0,0.05,14.40
7,National Grid PLC,NGG,NYSE,The mentioned news article describes a tragic ...,3.0,2024-02-17 05:15:00,"Virginia home explodes with fire crews inside,...",60.0,0.15,65.00
8,PG&E Corporation,PCG,NYSE,The forecasted extreme weather conditions in N...,3.0,2024-02-17 15:35:00,"Northern California forecast: Rounds of rain, ...",16.0,0.13,16.67
9,Utility Companies providing propane services,SPH,New York Stock Exchange (NYSE),The news article about a Virginia home explodi...,3.0,2024-02-17 18:28:00,"Virginia home explodes with fire crews inside,...",25.0,3.40,19.40


In [15]:
df['date'] = pd.to_datetime(df['date'])

In [20]:
    start = now - datetime.timedelta(hours=48)
    end = now - datetime.timedelta(hours=23)
    sell_candidates = df[(start < df['date']) & (df['date'] < end)]


In [21]:
sell_candidates

,stock,ticker,exchange,reasoning,action,date,data,strike_price,bid_price,current_price
7,National Grid PLC,NGG,NYSE,The mentioned news article describes a tragic ...,3.0,2024-02-17 05:15:00,"Virginia home explodes with fire crews inside,...",60.0,0.15,65.00
8,PG&E Corporation,PCG,NYSE,The forecasted extreme weather conditions in N...,3.0,2024-02-17 15:35:00,"Northern California forecast: Rounds of rain, ...",16.0,0.13,16.67
9,Utility Companies providing propane services,SPH,New York Stock Exchange (NYSE),The news article about a Virginia home explodi...,3.0,2024-02-17 18:28:00,"Virginia home explodes with fire crews inside,...",25.0,3.40,19.40


In [17]:
to_sell = df[(now - datetime.timedelta(hours=24) < df['date']) & (df['date'] < now - datetime.timedelta(hours=5))].iloc[0]

IndexError: single positional indexer is out-of-bounds

In [138]:
to_sell

stock                                       Colgate-Palmolive Company
ticker                                                             CL
exchange                                                         NYSE
reasoning           The news of Colgate-Palmolive facing a lawsuit...
action                                                            3.0
date                                              2024-02-07 00:28:26
data                Colgate-Palmolive must face lawsuit over tooth...
strike_price                                                     83.0
buy_price                                                        0.15
stock_buy_price                                                 84.35
sell_price                                                       0.25
stock_sell_price                                                84.35
Name: 3, dtype: object

In [103]:
sell_options = n2r.get_option_data(to_sell['ticker'])

Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data 

In [105]:
sell_options = News2ROI.parse_option_data(sell_options)

In [111]:
row = sell_options.set_index('strike_price').loc[to_sell['strike_price']]

In [133]:
df = df.rename({"bid_price": "buy_price"}, axis=1)

In [135]:
df = df.rename({"current_price": "stock_buy_price"}, axis=1)

In [141]:
df.loc[to_sell.name, 'sell_price'] = row['ask_price']

df.loc[to_sell.name, 'stock_sell_price'] = row['current_price']

df.loc[to_sell.name, 'roi'] = row['ask_price']/to_sell['buy_price']

In [142]:
df

,stock,ticker,exchange,reasoning,action,date,data,strike_price,buy_price,stock_buy_price,sell_price,stock_sell_price,roi
0,Vista Outdoor Inc.,VSTO,NYSE,The news about California's ammunition backgro...,5.0,2024-02-06 03:08:50,California ammunition background check law can...,27.5,0.05,29.95,NaN,NaN,NaN
1,Vista Outdoor Inc.,VSTO,NYSE,As a manufacturer and seller of ammunition and...,5.0,2024-02-06 03:08:50,California ammunition background check law can...,27.5,0.05,29.95,NaN,NaN,NaN
2,Vista Outdoor Inc.,VSTO,NYSE,The news about California's ammunition backgro...,5.0,2024-02-06 03:08:50,California ammunition background check law can...,27.5,0.05,29.95,NaN,NaN,NaN
3,Colgate-Palmolive Company,CL,NYSE,The news of Colgate-Palmolive facing a lawsuit...,3.0,2024-02-07 00:28:26,Colgate-Palmolive must face lawsuit over tooth...,83.0,0.15,84.35,0.25,84.35,1.666667
4,Colgate-Palmolive Company,CL,New York Stock Exchange (NYSE),The lawsuit against Colgate-Palmolive may have...,3.0,2024-02-07 00:28:26,Colgate-Palmolive must face lawsuit over tooth...,83.0,0.15,84.35,NaN,NaN,NaN
-1,Colgate-Palmolive Company,CL,New York Stock Exchange,The lawsuit regarding misleading recyclability...,4.0,2024-02-07 00:28:26,Colgate-Palmolive must face lawsuit over tooth...,83.0,0.15,84.35,NaN,NaN,NaN


In [ ]:
store_candidate(

In [113]:
to_sell

stock                                       Colgate-Palmolive Company
ticker                                                             CL
exchange                                                         NYSE
reasoning           The news of Colgate-Palmolive facing a lawsuit...
action                                                            3.0
date                                              2024-02-07 00:28:26
data                Colgate-Palmolive must face lawsuit over tooth...
strike_price                                                     83.0
bid_price                                                        0.15
current_price                                                   84.35
sell_price                                                       0.25
stock_sell_price                                                84.35
Name: 3, dtype: object

In [ ]:
df.loc[-1] = candidate
df = df.reset_index(drop=True)


In [151]:
def store_candidates(bucket_name, df):
    df['date'] = df['date'].dt.strftime('%Y-%m-%d %H:%M:%S')
    s_buf = BytesIO()
    df.to_csv(s_buf, sep='\t')
    s_buf.seek(0)
    s3.upload_fileobj(s_buf, bucket_name, "candidates.tsv")


In [161]:
store_candidates(bucket_name, df)

In [45]:
print(candidate.to_json(indent=2))

{
  "stock":"Colgate-Palmolive Company",
  "ticker":"CL",
  "exchange":"New York Stock Exchange",
  "reasoning":"The lawsuit regarding misleading recyclability claims may negatively impact Colgate-Palmolive's brand image and consumer trust. It could also lead to financial penalties or to increased costs if the company is required to adjust its packaging to be more environmentally friendly. There could be a short-term drop in stock price due to these factors, as investors react to potential legal fees, settlements, and the potential cost of rebranding or repackaging products. Additionally, ESG (environmental, social, and governance) considerations are increasingly important to investors, and negative news in this arena might affect investment decisions.",
  "action":4,
  "date":"2024-02-07 00:28:26.000000",
  "data":"Colgate-Palmolive must face lawsuit over toothpaste tube recycling claims - Colgate-Palmolive must face a lawsuit alleging it misleads consumers by claiming its plastic too

In [146]:
# j = {'account_sid': 'AC417be496c2958add9742abcc83840cb7',
#      'auth_token': '32e6cee11411c6037525b250c973eb2f',
#      'from': '+18662593587',
#      'to': '+14159002444'}


In [110]:
# with open('/root/.twilio.json', 'w') as f:
#     f.write(json.dumps(j))

In [111]:
# s3.upload_file("/root/.twilio.json", bucket_name, ".twilio.json")

In [46]:
n2r.notify(candidate)

'SMd2edc028d742480b4ec41eed8b151b26'